In [92]:
import tensorflow as  tf
from transformers import AutoModelForSeq2SeqLM

In [1]:
import pandas as pd

In [71]:
data=pd.read_csv("arabic.csv")

In [72]:
data

,Unnamed: 0,input,target
0,0,هو هنيك، قدام معلومات السياح بالضبط.,إنه هنك ، بالضبط قبل المعلومات السياحية.
1,1,مالي سمعان من قبل بهالعنوان هون.,مالي سيمون من قبل مع هذا العنوان.
2,2,روح ساوي لبين ما تشوف صيدلية.,روح ما تراه صيدلية.
3,3,بشقد الفطور؟,مع الفطور؟
4,4,شلون بقدر أساعدك؟,كيف يمكنني مساعدتك؟
...,...,...,...
1195,1195,خليني أحط كمادة باردة عشان يخف الوجع.,اسمحوا لي أن أضع مادة باردة لتقليل الألم.
1196,1196,هي بدها تشتري فستان.,إنها تريد شراء فستان.
1197,1197,كم بكلف التكسي لعند الفندق؟,كم تكلفة التاكسي للفندق؟
1198,1198,وين الحمام؟,أين الحمام؟


In [73]:
Input=data["input"].values

In [74]:
li=[i for  i in Input]

In [75]:
target=data["target"].values

In [76]:
target=[i for  i in target]

In [156]:
import torch
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

# Define your training and evaluation datasets
class CustomDataset(Dataset):
    def __init__(self, data, labels, tokenizer, max_length):
        self.data = data
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data[idx])
        label = str(self.labels[idx])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        encod = self.tokenizer.encode_plus(
            label,
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            #return_attention_mask=True,
            return_tensors='pt'
        )

        inputs = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': encod['labels'].flatten()
        }

        return inputs

In [157]:
from transformers import Trainer, TrainingArguments

In [158]:
import torch


In [159]:
# Define the configuration for the encoder-decoder model
#config = EncoderDecoderConfig.from_encoder_decoder_configs(encoder_config=BertConfig.from_pretrained(arabert_model_name),
                                                           #decoder_config=BertConfig.from_pretrained(arabert_model_name))

# Load the encoder-decoder model
#model = EncoderDecoderModel(config=config)

In [160]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv2")

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [161]:
train = tokenizer(li, truncation=True, padding=True)
#label = tokenizer(target, truncation=True, padding=True)
train_dataset = CustomDataset(li, target, tokenizer, 64)

In [162]:
train_dataset.max_length

64

In [163]:
train[0]

Encoding(num_tokens=55, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [164]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='epoch'
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    #eval_dataset=eval_dataset
)

In [165]:
# Train the model
trainer.train()

KeyError: 'labels'

In [ ]:
# Evaluate the model
trainer.evaluate()

In [80]:
# Tokenize and prepare the dataset
def tokenize_and_prepare_dataset(dataset):
    tokenized_inputs = tokenizer(dataset['Inputs'], padding=True, truncation=True, return_tensors="pt")
    tokenized_outputs = tokenizer(dataset['target'], padding=True, truncation=True, return_tensors="pt")
    return {k: tokenized_inputs[k] for k in tokenized_inputs}, {k: tokenized_outputs[k] for k in tokenized_outputs}

#train_dataset_tokenized = tokenize_and_prepare_dataset(dataset)
#validation_dataset_tokenized = tokenize_and_prepare_dataset(validation_dataset)

In [103]:
tokenized_inputs = tokenizer(li, padding=True, truncation=True, return_tensors="pt").input_ids
tokenized_outputs = tokenizer(target, padding=True, truncation=True, return_tensors="pt").input_ids

In [104]:
tokenized_outputs

tensor([[    2,  1749,  4526,  ...,     0,     0,     0],
        [    2,  5507, 21497,  ...,     0,     0,     0],
        [    2,  4435,   394,  ...,     0,     0,     0],
        ...,
        [    2,  1710,  8200,  ...,     0,     0,     0],
        [    2,  4799, 16793,  ...,     0,     0,     0],
        [    2,  2405, 22320,  ...,     0,     0,     0]])

In [88]:
train_dataset=tf.data.Dataset.from_tensor_slices((dict(tokenized_inputs),dict(tokenized_inputs)))

In [100]:
translated_ids = encoder_model.generate(Input[0])

TypeError: The current model class (BertModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'BertLMHeadModel'}

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

In [ ]:
trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:




# Fine-tune the model

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
